<a href="https://colab.research.google.com/github/Shrishml/transformers/blob/main/bart_t5_training_Shrish_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.19.3


Loading my dataset

test,train & valid

In [ ]:
split_list = ['train', 'test', 'valid']
for i in range(3):
  file1 = open('/content/drive/MyDrive/Transformers /data/'+split_list[i]+'_data_title_summary.txt', 'r')
  Lines = file1.readlines()
  file2 = open('/content/drive/MyDrive/Transformers /data/'+split_list[i]+'_data_title_summary.csv', 'w')
  file2.writelines(('data<s>summary\n'))
  for line in Lines:
    line = line[:-3]
    line= line+'\n'
    line = line.replace('\t', ' ')
    line_list = line.split('<s>')
    line = '<s>'.join([line_list[1]+line_list[0], line_list[2] ])
    file2.writelines(line)
  file2.close() 




In [ ]:
#latest
import pandas as pd
pd.read_csv('/content/drive/MyDrive/Transformers /data/test_data_title_summary.csv', sep='<s>', header = 0, engine='python')

,data,summary
0,'title': 'Countries with the lowest fertility ...,'summary': 'This statistic shows the countrie...
1,'title': 'Expenditure of affluent U.S. househo...,'summary': 'The statistic shows the expenditu...
2,'title': 'Quarterly average daily rate in hote...,'summary': 'This statistic shows the quarterl...
3,'title': 'Population age structure in metropol...,'summary': 'This graph shows the age structur...
4,'title': 'Average retail price for white sugar...,'summary': 'The average retail price for two ...
...,...,...
1217,'title': 'Foster care in the U.S. - number of ...,'summary': 'This statistic shows the number o...
1218,'title': 'Global automotive industry : capacit...,'summary': 'This statistic represents the cap...
1219,'title': 'Capacity factor of nuclear power pla...,'summary': 'This statistic represents the cap...
1220,'title': 'Job creation by U.S. president from ...,'summary': 'This statistic shows the monthly ...


In [ ]:
from datasets import load_dataset
data_files={"train":"/content/drive/MyDrive/Transformers /data/train_data_title_summary.csv", "test":"/content/drive/MyDrive/Transformers /data/test_data_title_summary.csv",
            "valid":"/content/drive/MyDrive/Transformers /data/valid_data_title_summary.csv" } 
local_ctt_dataset=load_dataset("csv",data_files=data_files, sep='<s>')
local_ctt_dataset

Using custom data configuration default-c353991fe55c6dd0


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c353991fe55c6dd0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['data', 'summary'],
        num_rows: 5703
    })
    test: Dataset({
        features: ['data', 'summary'],
        num_rows: 1222
    })
    valid: Dataset({
        features: ['data', 'summary'],
        num_rows: 1222
    })
})

if 'None'

In [ ]:
local_ctt_dataset = local_ctt_dataset.filter(lambda x: (x['summary'] is not None))

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Data pre-processing

In [ ]:
model_checkpoint = "t5-small"
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:

if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["data"]]
    # print(inputs)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)
    #print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(local_ctt_dataset['train'][1:3])

{'input_ids': [[21603, 10, 3, 31, 21869, 31, 10, 3, 31, 3183, 226, 15, 26, 19276, 1396, 7644, 1080, 1360, 3, 6, 57, 1719, 31, 3, 31, 6757, 31, 10, 3, 31, 10628, 651, 2648, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 3, 22776, 6993, 9217, 834, 134, 4748, 7, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 3, 20677, 6993, 3826, 834, 184, 834, 25480, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 9264, 591, 6993, 1150, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 209, 27336, 6993, 3, 21134, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 9997, 927, 6993, 37, 834, 28746, 7, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 204, 24273, 6993, 1740, 13002, 834, 883, 834, 2915, 834, 77, 29884, 7, 220, 22493, 3, 31, 1], [21603, 10, 3, 31, 21869, 31, 10, 3, 31, 188, 5291, 3, 18, 2699, 3199, 4407, 12, 846, 31, 3, 31, 6757, 31, 10, 3, 31, 476, 2741, 846, 12772, 834, 77, 834, 17030, 834, 18985, 3097, 3166, 2929, 1233, 12772, 834, 77, 834, 17030, 834, 18985, 9668, 4450, 2929, 1421, 12772, 834, 77, 83

In [ ]:
local_ctt_dataset['train'][:2]

{'data': ["'title': 'Global spending on motorsports sponsorships 2011 to 2017' 'data': 'Year 2017 Spending_in_billion_U.S._dollars 5.75 Year 2016 Spending_in_billion_U.S._dollars 5.58 Year 2015 Spending_in_billion_U.S._dollars 5.43 Year 2014 Spending_in_billion_U.S._dollars 5.26 Year 2013 Spending_in_billion_U.S._dollars 5.12 Year 2012 Spending_in_billion_U.S._dollars 4.97 Year 2011 Spending_in_billion_U.S._dollars 4.83 ' ",
  "'title': 'Fixed broadband internet subscription rate 2019 , by region' 'data': 'Country Africa Rate_per_100_inhabitants 0.4 Country Arab_States Rate_per_100_inhabitants 8.1 Country Asia_&_Pacific Rate_per_100_inhabitants 14.4 Country World Rate_per_100_inhabitants 14.9 Country CIS Rate_per_100_inhabitants 19.8 Country The_Americas Rate_per_100_inhabitants 22.0 Country Europe Rate_per_100_inhabitants 31.9 ' "],
 'summary': [" 'summary': 'This statistic shows the worldwide spending for motorsports sponsorship from 2011 to 2017 . In the year 2013 , the global motor

In [ ]:
local_ctt_dataset['train'][0]

{'data': "'title': 'Global spending on motorsports sponsorships 2011 to 2017' 'data': 'Year 2017 Spending_in_billion_U.S._dollars 5.75 Year 2016 Spending_in_billion_U.S._dollars 5.58 Year 2015 Spending_in_billion_U.S._dollars 5.43 Year 2014 Spending_in_billion_U.S._dollars 5.26 Year 2013 Spending_in_billion_U.S._dollars 5.12 Year 2012 Spending_in_billion_U.S._dollars 4.97 Year 2011 Spending_in_billion_U.S._dollars 4.83 ' ",
 'summary': " 'summary': 'This statistic shows the worldwide spending for motorsports sponsorship from 2011 to 2017 . In the year 2013 , the global motorsports sponsorship spending amounted to 5.12 billion U.S. dollars .' <"}

In [ ]:
tokenized_datasets = local_ctt_dataset.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Fine tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 6
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-CTT",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
import nltk
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    #test_dataset=tokenized_datasets["test"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [ ]:
tokenized_datasets["test"]

Dataset({
    features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1222
})

In [ ]:
trainer.train()

# cuda out of memory error
# 1. bacasue of more input lendth and output length = make it input = 1024 and output = 128, while tokenization
# 2. will be able to train t5 small only , not base or large

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: data, summary. If data, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5703
  Num Epochs = 4
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 3804


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.077500,1.762770,29.040900,18.077500,27.281000,27.982200,19.000000
2,1.953000,1.705509,29.130300,18.412300,27.414900,28.175900,19.000000
3,1.891000,1.677923,29.153700,18.481200,27.444500,28.188100,19.000000
4,1.855700,1.669063,29.145000,18.490300,27.453700,28.209400,19.000000


Saving model checkpoint to t5-small-finetuned-CTT/checkpoint-500
Configuration saved in t5-small-finetuned-CTT/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-CTT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-CTT/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-CTT/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: data, summary. If data, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1222
  Batch size = 6
Saving model checkpoint to t5-small-finetuned-CTT/checkpoint-1000
Configuration saved in t5-small-finetuned-CTT/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-CTT/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-f

TrainOutput(global_step=3804, training_loss=1.932978841659273, metrics={'train_runtime': 1241.4117, 'train_samples_per_second': 18.376, 'train_steps_per_second': 3.064, 'total_flos': 3921333086453760.0, 'train_loss': 1.932978841659273, 'epoch': 4.0})

Testing the model

In [ ]:
model_retrained = AutoModelForSeq2SeqLM.from_pretrained('/content/t5-small-finetuned-CTT/checkpoint-3500')

loading configuration file /content/t5-small-finetuned-CTT/checkpoint-3500/config.json
Model config T5Config {
  "_name_or_path": "/content/t5-small-finetuned-CTT/checkpoint-3500",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
 

In [ ]:
sample_text1=["year|McDonald's_CR(@mcdonaldscr)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|44.91|y|line_chart year|Pizza_Hut_CR(@PizzaHutCRC)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|42.28|y|line_chart year|Taco_Bell_Costa_Rica(@tacobell_cr)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|16.62|y|line_chart year|Subway_Costa_Rica(@subwaycr)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|10.53|y|line_chart year|Starbucks_Costa_Rica(@Starbucks_cr)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|5.64|y|line_chart year|HRC_San_Jose_CR(@HardRockCafeCR)|x|line_chart Retail_food_brands_in_Costa_Rica_with_the_largest_number_of_Twitter_followers_as_of_May_2021_(in_1,000s)|3.79|y|line_chart"]

In [ ]:
test_text2=[" 'title':	'Countries with the lowest fertility rate globally 2050 to 2055'<s> 'data':	'Country Singapore Average_number_of_children_per_woman 1.38 Country Republic_of_Korea Average_number_of_children_per_woman 1.44 Country United_Arab_Emirates Average_number_of_children_per_woman 1.45 Country Puerto_Rico Average_number_of_children_per_woman 1.45 Country Bosnia_and_Herzegovina Average_number_of_children_per_woman 1.47 Country Saint_Lucia Average_number_of_children_per_woman 1.48 Country Greece Average_number_of_children_per_woman 1.5 Country Cyprus Average_number_of_children_per_woman 1.51 Country Italy Average_number_of_children_per_woman 1.51 Country Republic_of_Moldova Average_number_of_children_per_woman 1.52 Country China_Taiwan_Province_of_China Average_number_of_children_per_woman 1.53 Country Albania Average_number_of_children_per_woman 1.53 Country Mauritius Average_number_of_children_per_woman 1.54 Country Thailand Average_number_of_children_per_woman 1.54 Country Qatar Average_number_of_children_per_woman 1.56 Country Nepal Average_number_of_children_per_woman 1.56 Country Croatia Average_number_of_children_per_woman 1.56 Country Japan Average_number_of_children_per_woman 1.57 Country Serbia Average_number_of_children_per_woman 1.57 Country Brazil Average_number_of_children_per_woman 1.58 Country North_Macedonia Average_number_of_children_per_woman 1.58 Country Brunei_Darussalam Average_number_of_children_per_woman 1.59 Country Portugal Average_number_of_children_per_woman 1.59 Country Spain Average_number_of_children_per_woman 1.59 Country Canada Average_number_of_children_per_woman 1.59 ' "]

In [ ]:
summaries={}

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

In [ ]:
from transformers import pipeline
pipe = pipeline("summarization", model="/content/t5-small-finetuned-CTT/checkpoint-3500")
pipe_out = pipe(test_text2)
summaries["my_model"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

loading configuration file /content/t5-small-finetuned-CTT/checkpoint-3500/config.json
Model config T5Config {
  "_name_or_path": "/content/t5-small-finetuned-CTT/checkpoint-3500",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
 

In [ ]:
summaries["bart"]

'x|line_chart Retail food brands in CostaRica with the largest number of Twitter followers as of May 2021(in_1,000s)|44.91|y|line-chart year|PizzaHutCRC .\nRetail food brands in the United States are the largest among Twitter followers in the world .\nIn the United Kingdom , a large number of Instagram followers have a Twitter account .'

In [ ]:
test_sum2 = "'summary':	'This statistic shows the countries and territories with the lowest projected fertility rate between 2050 and 2055 . Between 2050 and 2055 , Singapore is projected to have the lowest fertility rate , with an average of 1.38 children born per woman .'"

In [ ]:
summaries["my_model"]

"'summary': 'This statistic shows the countries with the lowest fertility rate globally from 2050 to 2055 .\nIn 2018 , Singapore 's average fertility rate was 1.38 children per woman .'"

In [ ]:
summaries={}
for i in range(1200):
  summaries[i] = {}
  test_text = local_ctt_dataset['test'][i]['data']
  pipe_out = pipe(test_text)
  summaries[i]["original"] = local_ctt_dataset['test'][i]['summary']
  summaries[i]["my_model"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
  # print(summaries[i]["original"], '\n\n', summaries[i]["my_model"], '\n\n')

Your max_length is set to 200, but you input_length is only 131. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Your max_length is set to 200, but you input_length is only 89. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 200, but you input_length is only 190. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)
Your max_length is set to 200, but you input_length is only 158. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=79)
Your max_length is set to 200, but you input_length is only 144. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=72)
Your max_length is set to 200, but you input_length is only 163. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=81)
Your max_length is set to 200, but you input_length is only 159. You might co

KeyboardInterrupt: ignored

Original t5 small model

In [ ]:
df = pd.DataFrame(summaries).T
df

,original,my_model
0,'summary': 'This statistic shows the countrie...,'summary': 'This statistic shows the countries...
1,'summary': 'The statistic shows the expenditu...,'summary': 'This statistic shows the expenses ...
2,'summary': 'This statistic shows the quarterl...,'summary': 'This statistic shows the average d...
3,'summary': 'This graph shows the age structur...,'summary': 'This statistic shows the populatio...
4,'summary': 'The average retail price for two ...,'summary': 'This statistic shows the average r...
...,...,...
207,'summary': 'This graphic displays the market ...,'summary': 'This statistic shows the global ph...
208,'summary': 'This statistic shows the number o...,'summary': 'This statistic shows the number of...
209,'summary': 'The statistic shows the average c...,'summary': 'This statistic shows the average s...
210,'summary': 'Net loss attributable to Tesla 's...,'summary': 'This statistic shows Tesla 's net ...


In [ ]:
df.to_csv('/content/drive/MyDrive/Transformers /test_t5_small.csv')

In [ ]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 26.3 MB/s 
     |████████████████████████████████| 1.2 MB 8.0 MB/s 
     |████████████████████████████████| 132 kB 25.5 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 8.9 MB 59.6 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_version)
model = AutoModel.from_pretrained(model_version, output_attentions=True)

Didn't find file /content/t5-small-finetuned-CTT/checkpoint-3500/spiece.model. We won't load it.
Didn't find file /content/t5-small-finetuned-CTT/checkpoint-3500/added_tokens.json. We won't load it.
loading file None
loading file /content/t5-small-finetuned-CTT/checkpoint-3500/tokenizer.json
loading file None
loading file /content/t5-small-finetuned-CTT/checkpoint-3500/special_tokens_map.json
loading file /content/t5-small-finetuned-CTT/checkpoint-3500/tokenizer_config.json
loading configuration file /content/t5-small-finetuned-CTT/checkpoint-3500/config.json
Model config T5Config {
  "_name_or_path": "/content/t5-small-finetuned-CTT/checkpoint-3500",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions"

In [ ]:
encoder_input_ids = tokenizer(test_text2[0], return_tensors="pt", add_special_tokens=True).input_ids
decoder_input_ids = tokenizer(test_sum2, return_tensors="pt", add_special_tokens=True).input_ids

outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

In [ ]:
from bertviz import model_view
model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens = decoder_text)

In [ ]:
from bertviz import model_view
head_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens = decoder_text)

In [ ]:
# ! cp -r '/content/t5-small-finetuned-CTT' '/content/drive/MyDrive/Transformers /model'

In [ ]:
#from transformers import pipeline
pipe2 = pipeline("summarization", model="t5-small")
pipe_out = pipe2(test_text2)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_lengt

In [ ]:
summaries["t5"]

'\'title\': \'Countries with the lowest fertility rate globally 2050 to 2055\'s> \'data\': "country Singapore Average_number_of_children_per_woman 1.38 Country Republic_of-Korea Average_nine_death_child_per-woman 1.44 Country United_Arab_Emirates Average_nomber_distribution 1.45 Country Bosnia_and_Herzegogovina Average_name_delath_de_children'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/t5-small-finetuned-CTT' '/content/drive/MyDrive/Projects/NLG'

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer =Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
# !cp '/content/valid_data_title_summary.csv' '/content/drive/MyDrive/Projects/NLG'